In [1]:
import pandas as pd
import datetime as dt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
events = pd.read_csv('fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', low_memory=False)

In [35]:
train_data = pd.read_csv('train_data.csv',index_col=0)

In [36]:
test_kaggle = pd.read_csv('test_kaggle.csv',index_col=0)

In [6]:
events['week'] = pd.to_datetime(events['timestamp']).dt.week

In [7]:
events['week'].value_counts()

21    540477
20    495049
22    408822
19    166141
18    118139
17     96538
16     67727
15     65513
14     60947
13     55405
12     55377
11     40157
10     31867
9      28420
8      21058
5      16097
7      15546
6      15323
3      12688
2      12122
4      11537
1       6731
Name: week, dtype: int64

In [8]:
#SEPARO EN PERIODOS DE DOS SEMANAS
def periodo (week):
    return (week+1)//2

events['periodo'] = events['week'].apply(lambda week: periodo(week))

In [7]:
events['periodo'].value_counts()

11    949299
10    661190
9     214677
8     133240
7     116352
6      95534
5      60287
4      36604
3      31420
2      24225
1      18853
Name: periodo, dtype: int64

# Feature: Cantidad de eventos por usuario 

Solo trabajo con alguno de los periodos

In [126]:
eventos = events.loc[events['periodo']>6]

In [127]:
eventos= eventos[['person','periodo','event','timestamp']]

In [134]:
eventos_por_periodo = eventos.groupby(['periodo','person','event']).count().unstack()

In [135]:
eventos_por_periodo.reset_index(inplace=True)

In [138]:
eventos_por_periodo.set_index('periodo',inplace=True)

In [139]:
eventos_por_periodo.columns = [col[1] for col in eventos_por_periodo.columns]

In [141]:
eventos_por_periodo.reset_index(inplace=True)

In [142]:
eventos_por_periodo.rename(columns={'':'person'},inplace=True)
eventos_por_periodo.fillna(value=0,inplace=True)

In [144]:
eventos_por_periodo.columns = [col+'_mean' for col in eventos_por_periodo.columns]

In [146]:
eventos_por_periodo.rename(columns={'periodo_mean':'periodo','person_mean':'person'},inplace=True)

In [18]:
eventos_por_periodo.rename(columns={7:'eventos_periodo_7',8:'eventos_periodo_8',9:'eventos_periodo_9',10:'eventos_periodo_10',11:'eventos_periodo_11'}, inplace=True)

In [150]:
eventos_por_periodo_mean = eventos_por_periodo.groupby(['person','periodo']).mean()

In [152]:
eventos_por_periodo_mean.reset_index(inplace=True)

In [154]:
eventos_por_periodo_mean = eventos_por_periodo_mean.groupby('person').mean().reset_index()

In [254]:
train_data = train_data.merge(eventos_por_periodo_mean,on='person',how='left')
test_kaggle = test_kaggle.merge(eventos_por_periodo_mean,on='person',how='left')

In [256]:
test_kaggle.fillna(value=0,inplace=True)

# Gradient boosting

In [19]:
gBoosting = GradientBoostingClassifier(n_estimators=20, max_features=4,max_depth=10)

In [ ]:
X = train_data[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']]
Y = train_data['label']

X_train, X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3)

In [ ]:
gBoosting.fit(X_train,Y_train)

In [ ]:
gBoosting.score(X_test,Y_test)

In [ ]:
test_kaggle['prediction'] = gBoosting.predict(test_kaggle[['eventos_periodo_7', 'eventos_periodo_8','eventos_periodo_9', 'eventos_periodo_10', 'eventos_periodo_11']])

In [ ]:
test

In [ ]:
submit_kaggle = pd.DataFrame({'person':test_kaggle['person'],'label':test_kaggle['prediction']})
submit_kaggle.reset_index(inplace = True)
submit_kaggle.drop(columns='index',inplace=True)
submit_kaggle.set_index(keys='person',inplace=True)

In [ ]:
submit_kaggle.to_csv('submit_kaggle.csv')

# Feature: region - resolucion de pantalla

In [164]:
region = events[['person','region','screen_resolution','periodo']].dropna()
region

,person,region,screen_resolution,periodo
2136629,4640420b,Parana,320x570,10
2136630,4640420b,Parana,320x570,10
2136631,4640420b,Unknown,1280x1024,10
2136632,4640420b,Parana,800x1280,10
2136633,4640420b,Parana,800x1280,10
2136634,4640420b,Parana,320x570,10
2136635,4640420b,Parana,800x1280,10
2136636,4640420b,Sao Paulo,320x570,10
2136637,4640420b,Unknown,1280x1024,10
2136638,4640420b,Parana,800x1280,11


In [26]:
region = region.loc[region['periodo']>6]
region['region_resolucion'] = region['region']+region['screen_resolution']
region_resolucion = region[['person','region_resolucion']]
region_resolucion

/home/aaron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,region_resolucion
2136629,4640420b,Parana320x570
2136630,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136633,4640420b,Parana800x1280
2136634,4640420b,Parana320x570
2136635,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136637,4640420b,Unknown1280x1024
2136638,4640420b,Parana800x1280


In [27]:
region_resolucion = region_resolucion.drop_duplicates()
region_resolucion

,person,region_resolucion
2136629,4640420b,Parana320x570
2136631,4640420b,Unknown1280x1024
2136632,4640420b,Parana800x1280
2136636,4640420b,Sao Paulo320x570
2136640,4640420b,Parana320x522
2136652,4640420b,Santa Catarina1280x1024
2136655,4640420b,Santa Catarina320x522
2136659,4640420b,Sao Paulo1280x1024
2136660,f35dddc8,Piaui360x640
2136661,c82ecb12,Rio de Janeiro360x640


In [24]:
region_resolucion['region_resolucion_h'] = region_resolucion['region_resolucion'].apply(lambda a: hash(a))
region_resolucion

/home/aaron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,person,region_resolucion,region_resolucion_h
2136629,4640420b,Parana320x570,-3766738917691135549
2136631,4640420b,Unknown1280x1024,7496655169086332573
2136632,4640420b,Parana800x1280,-3613383972500808549
2136636,4640420b,Sao Paulo320x570,7685767176648590612
2136640,4640420b,Parana320x522,-4105794089159026561
2136652,4640420b,Santa Catarina1280x1024,-6237622606702027072
2136655,4640420b,Santa Catarina320x522,-7704385953323940971
2136659,4640420b,Sao Paulo1280x1024,-8148664137530837375
2136660,f35dddc8,Piaui360x640,-3084035898434772204
2136661,c82ecb12,Rio de Janeiro360x640,-4792852114531465108


In [39]:
train_data = train_data.merge(region_resolucion,on='person',how='left')

In [ ]:
train_data.to_csv('features/train_data.csv')

In [ ]:
test_kaggle = test_kaggle.merge(region_resolucion,on='person',how='left')

In [ ]:
test_kaggle.to_csv('test_kaggle.csv')

# Feature : Condicion del producto

In [169]:
condicion_general = events[['person','event','condition','periodo']]
condicion_general = condicion_general.loc[condicion_general['periodo']>6]

In [170]:
condicion_general = condicion_general.groupby(['periodo','person','condition']).count().unstack(level=2)

In [171]:
condicion_general.reset_index(inplace=True)
condicion_general.fillna(value=0,inplace=True)

In [172]:
condicion_general.set_index('periodo',inplace=True)

In [173]:
condicion_general.columns = [col[1] for col in condicion_general.columns]

In [174]:
condicion_general.reset_index(inplace=True)

In [175]:
condicion_general.rename(columns={'':'person'}, inplace=True)

In [176]:
condicion_general_mean = condicion_general.groupby(['person','periodo']).mean()

In [177]:
condicion_general_mean.reset_index(inplace=True)

In [178]:
condicion_general_mean = condicion_general_mean.drop(columns = 'periodo')

In [179]:
condicion_general_mean_por_periodo = condicion_general_mean.groupby('person').mean()

In [257]:
train_data = train_data.merge(condicion_general_mean_por_periodo,on='person',how='left')

In [258]:
train_data.fillna(value=0,inplace=True)
train_data.to_csv('train_data.csv')

In [259]:
test_kaggle = test_kaggle.merge(condicion_general_mean_por_periodo,on='person',how='left')

In [260]:
test_kaggle.fillna(value = 0, inplace=True)

In [262]:
test_kaggle.to_csv('test_kaggle.csv')

# Feature: Promedio de ingresos al sitio 

In [190]:
visitas = events.loc[events['event']=='visited site']

In [191]:
visitas = visitas[['timestamp','person','event','periodo']]
visitas = visitas[visitas['periodo']>6]

In [192]:
visitas['timestamp'] = pd.to_datetime(visitas['timestamp'])
visitas['dia'] = visitas['timestamp'].dt.dayofweek

In [199]:
visitas_por_dia = visitas.groupby(['periodo','person','dia'])['event'].count().unstack()

In [203]:
visitas_por_dia.fillna(value=0,inplace=True)
visitas_por_dia.reset_index(inplace=True)

In [206]:
visitas_por_dia_mean = visitas_por_dia.groupby(['person','periodo']).mean()

In [208]:
visitas_por_dia_mean.reset_index(inplace=True)

In [213]:
visitas_por_dia_mean.drop(columns = 'periodo',inplace=True)
visitas_por_dia_mean = visitas_por_dia_mean.groupby('person').mean()

In [215]:
visitas_por_dia_mean.reset_index(inplace=True)

In [217]:
visitas_por_dia_mean.rename(columns={0:'visitas_dom',1:'visitas_lun',2:'visitas_mar',3:'visitas_mier',4:'visitas_jue',5:'visitas_vier',6:'visitas_sab'},inplace=True)

In [263]:
train_data = train_data.merge(visitas_por_dia_mean,on='person',how='left')
test_kaggle = test_kaggle.merge(visitas_por_dia_mean,on='person',how='left')

In [264]:
train_data.fillna(value=0,inplace=True)
test_kaggle.fillna(value=0,inplace=True)

In [265]:
train_data.to_csv('train_data.csv')
test_kaggle.to_csv('test_kaggle.csv')

# Feature: Cantidad promedio de compras por usuario por periodo

In [225]:
conversiones = events[['timestamp','person','event','periodo']]
conversiones = conversiones.loc[conversiones['event']=='conversion']
conversiones['timestamp'] = pd.to_datetime(conversiones['timestamp'])
conversiones['mes'] = conversiones['timestamp'].dt.month

In [227]:
conversiones_por_periodo = conversiones.groupby(['periodo','person'])['event'].count().to_frame()
conversiones_por_periodo.reset_index(inplace=True)
conversiones_por_periodo.fillna(0)

,periodo,person,event
0,1,027870af,1
1,1,064ec6ea,1
2,1,08343c0e,1
3,1,0a5879fc,1
4,1,11e3fcc3,2
5,1,123fde23,1
6,1,29e1938c,1
7,1,343618be,1
8,1,3575e80c,2
9,1,40f9b318,1


In [228]:
conversiones_por_periodo['mean_conversion'] = conversiones_por_periodo['event']/11
conversiones_por_periodo

,periodo,person,event,mean_conversion
0,1,027870af,1,0.090909
1,1,064ec6ea,1,0.090909
2,1,08343c0e,1,0.090909
3,1,0a5879fc,1,0.090909
4,1,11e3fcc3,2,0.181818
5,1,123fde23,1,0.090909
6,1,29e1938c,1,0.090909
7,1,343618be,1,0.090909
8,1,3575e80c,2,0.181818
9,1,40f9b318,1,0.090909


In [230]:
conversiones_por_periodo_mean = conversiones_por_periodo.groupby('person').agg({'event':'mean'})
conversiones_por_periodo_mean.reset_index(inplace = True)

In [233]:
conversiones_por_periodo_mean.rename(columns={'event':'conversiones_mean'}, inplace=True)

In [266]:
train_data = train_data.merge(conversiones_por_periodo_mean,on='person',how='left')

In [267]:
test_kaggle = test_kaggle.merge(conversiones_por_periodo_mean,on='person',how='left')

In [268]:
train_data.fillna(value=0,inplace=True)
test_kaggle.fillna(value=0,inplace=True)

In [271]:
train_data.to_csv('train_data.csv')
test_kaggle.to_csv('test_kaggle.csv')

# Feature : Personas que vieron el mismo producto mas de 5 veces en el mismo periodo

In [288]:
viewed_product = events.loc[events['event']== 'viewed product']
viewed_product = viewed_product[viewed_product['periodo']>6]
cantidad_de_visitas_a_producto = viewed_product.groupby(['periodo','person','model']).agg({'timestamp':'count'})
cantidad_de_visitas_a_producto.reset_index(inplace=True)

In [290]:
cantidad_de_visitas_a_producto['vio_mas_de_5_veces'] = cantidad_de_visitas_a_producto['timestamp']>5

In [292]:
usuarios_vieron_mas_de_5_veces = cantidad_de_visitas_a_producto[cantidad_de_visitas_a_producto['vio_mas_de_5_veces']==True]

In [294]:
usuarios_vieron_mas_de_5_veces.drop_duplicates('person',inplace=True)

/home/aaron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [296]:
usuarios_vieron_mas_de_5_veces = usuarios_vieron_mas_de_5_veces[['person','vio_mas_de_5_veces']]

In [298]:
train_data = train_data.merge(usuarios_vieron_mas_de_5_veces,on='person',how='left')

In [302]:
train_data['vio_mas_de_5_veces'].replace(to_replace=0,value=False,inplace=True)

In [304]:
test_kaggle = test_kaggle.merge(usuarios_vieron_mas_de_5_veces,on='person',how='left')
test_kaggle['vio_mas_de_5_veces'].fillna(False,inplace=True)

In [306]:
test_kaggle.to_csv('test_kaggle.csv')
train_data.to_csv('train_data.csv')

# Feature : Colores de los dispositivos por periodo

In [388]:
color = events[['person','color','event','periodo']]

In [389]:
color = color.loc[color['event']=='viewed product']

In [390]:
color = color.groupby(['periodo','person','color'])['event'].count().unstack(level=0)

In [391]:
color.reset_index(inplace=True)

In [392]:
color = color.loc[(color['color'] == 'Dourado') | (color['color'] == 'Preto') | (color['color']== 'Branco')]
color = color.fillna(0)

In [393]:
for x in range(1,12):
    color[x] = color[x].apply(lambda a: 1 if a != 0 else a)
color

periodo,person,color,1,2,3,4,5,6,7,8,9,10,11
5,00091926,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,00091926,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
13,00091926,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
21,00091a7a,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
25,000ba417,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
32,000ba417,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
34,000ba417,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
46,000e4d9e,Branco,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
48,000e4d9e,Dourado,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
53,000e4d9e,Preto,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [394]:
color = color[['person',7,8,9,10,11]]
#color.replace(to_replace=1.0,value=None,inplace=True)

In [396]:
color['vio_color_mas_vendido'] = ((color[7]==1)|(color[8]==1)|(color[9]==1)|(color[10]==1)|(color[11]==1))

/home/aaron/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [408]:
usuarios_vieron_color_mas_vendido = color.loc[color['vio_color_mas_vendido']==True]
usuarios_vieron_color_mas_vendido = usuarios_vieron_color_mas_vendido[['person','vio_color_mas_vendido']]

In [409]:
train_data=train_data.merge(usuarios_vieron_color_mas_vendido,on='person',how='left')
train_data['vio_color_mas_vendido'].fillna(False,inplace=True)

In [411]:
test_kaggle = test_kaggle.merge(usuarios_vieron_color_mas_vendido,on='person',how='left')
test_kaggle['vio_color_mas_vendido'].fillna(False,inplace=True)

In [415]:
test_kaggle.drop_duplicates('person',inplace=True)

In [416]:
train_data.to_csv('train_data.csv')
test_kaggle.to_csv('test_kaggle.csv')

# Feature: Usuario solicito lista de productos de la misma marca

In [413]:
brand_listing = events[['person','event','periodo']]

In [82]:
brand_listing = brand_listing.loc[brand_listing['event']=='brand listing']

In [83]:
brand_listing = brand_listing.groupby(['periodo', 'person'])['event'].count().unstack(level=0)
brand_listing.reset_index(inplace=True)
brand_listing = brand_listing.fillna(0)
brand_listing

periodo,person,1,2,3,4,5,6,7,8,9,10,11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,9.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,14.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0
5,0019e639,0.0,17.0,17.0,0.0,0.0,0.0,0.0,0.0,33.0,66.0,32.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0


In [84]:
for x in range(1,12):
    brand_listing[x] = brand_listing[x].apply(lambda a: 1 if a != 0 else a)
brand_listing

periodo,person,1,2,3,4,5,6,7,8,9,10,11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0019e639,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [85]:
brand_listing.drop_duplicates(subset = 'person', inplace=True)

brand_listing.rename(columns={1:'visito_brand_listing_1',
                                              2:'visito_brand_listing_2',
                                              3:'visito_brand_listing_3',
                                              4:'visito_brand_listing_4',
                                              5:'visito_brand_listing_5',
                                              6:'visito_brand_listing_6',
                                              7:'visito_brand_listing_7',
                                              8:'visito_brand_listing_8',
                                              9:'visito_brand_listing_9',
                                              10:'visito_brand_listing_10',
                                              11:'visito_brand_listing_11',
                                              },inplace=True)

brand_listing = brand_listing.fillna(0)
brand_listing

periodo,person,visito_brand_listing_1,visito_brand_listing_2,visito_brand_listing_3,visito_brand_listing_4,visito_brand_listing_5,visito_brand_listing_6,visito_brand_listing_7,visito_brand_listing_8,visito_brand_listing_9,visito_brand_listing_10,visito_brand_listing_11
0,00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0019e639,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
6,001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
8,001e9aea,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,0027574e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


# Feature: Cantidad de conversiones por horario

In [3]:
def franja_horaria_de(hora):
    if hora.hour >= 19:
        return 'noche'
    if hora.hour >= 12:
        return 'tarde'
    if hora.hour >= 6:
        return 'maniana'
    return 'madrugada'

horarios_de_conversion = events[['person','event','timestamp']]
horarios_de_conversion = horarios_de_conversion.loc[horarios_de_conversion['event']=='conversion']
horarios_de_conversion['timestamp'] = pd.to_datetime(horarios_de_conversion['timestamp'])
horarios_de_conversion['hora'] = horarios_de_conversion['timestamp'].apply(franja_horaria_de)
horarios_de_conversion


,person,event,timestamp,hora
195,49c19e32,conversion,2018-05-18 01:12:20,madrugada
234,49c19e32,conversion,2018-05-18 01:05:23,madrugada
724,39df97e0,conversion,2018-05-29 07:51:14,maniana
744,380c0e60,conversion,2018-05-29 07:13:30,maniana
1198,35ee0cc8,conversion,2018-05-29 19:31:46,noche
2124,c5a0bc36,conversion,2018-05-29 18:36:30,tarde
2550,6f0826b2,conversion,2018-05-29 17:26:11,tarde
3263,576d1af6,conversion,2018-05-29 17:20:21,tarde
3391,04b13fd2,conversion,2018-05-29 19:02:08,noche
3558,bf15584a,conversion,2018-05-29 18:46:21,tarde


In [4]:
horarios_de_conversion = horarios_de_conversion.groupby(['hora', 'person'])['event'].count().unstack(level=0)
horarios_de_conversion.reset_index(inplace=True)
horarios_de_conversion = horarios_de_conversion.fillna(0)
horarios_de_conversion

hora,person,madrugada,maniana,noche,tarde
0,000ba417,0.0,0.0,0.0,1.0
1,001001be,0.0,0.0,1.0,0.0
2,001804a2,0.0,0.0,0.0,1.0
3,0019e639,0.0,0.0,1.0,1.0
4,001b0bf9,0.0,0.0,0.0,1.0
5,0020152e,0.0,0.0,0.0,1.0
6,002aea56,0.0,1.0,0.0,3.0
7,002ed810,0.0,0.0,2.0,0.0
8,0038a117,0.0,0.0,0.0,1.0
9,0043a48e,2.0,0.0,1.0,0.0


# Feature: porcentaje de productos vistos por conversion

In [83]:
prod_vistos_por_conversion = events[['person','event','timestamp','periodo']]
prod_vistos_por_conversion = prod_vistos_por_conversion.loc[(prod_vistos_por_conversion['event']=='conversion') | (prod_vistos_por_conversion['event']=='viewed product')]
prod_vistos_por_conversion

,person,event,timestamp,periodo
0,4886f805,viewed product,2018-05-18 00:11:59,10
1,ad93850f,viewed product,2018-05-18 00:11:27,10
2,0297fc1e,viewed product,2018-05-18 00:11:16,10
3,2d681dd8,viewed product,2018-05-18 00:11:14,10
4,cccea85e,viewed product,2018-05-18 00:11:09,10
6,1b9f7cf6,viewed product,2018-05-18 00:44:14,10
7,29ebb414,viewed product,2018-05-18 00:44:02,10
8,de8fe91b,viewed product,2018-05-18 00:43:59,10
10,99abca5a,viewed product,2018-05-18 00:42:14,10
11,3dc1950f,viewed product,2018-05-18 00:41:00,10


In [84]:
prod_vistos_por_conversion = prod_vistos_por_conversion.groupby(['periodo','person'])['event'].value_counts().unstack(level=2)
prod_vistos_por_conversion = prod_vistos_por_conversion.fillna(0)
prod_vistos_por_conversion

event             conversion  viewed product
periodo person                              
1       00546f4c         0.0             8.0
        006cabc5         0.0            36.0
        009944e1         0.0            24.0
        016b86b9         0.0            23.0
        01716e05         0.0             1.0
        01f061eb         0.0            14.0
        022ca64b         0.0            12.0
        027870af         1.0            28.0
        0297fc1e         0.0            11.0
        02a11379         0.0             1.0
        02ac4fca         0.0             1.0
        0318fd8d         0.0             5.0
        03439225         0.0            33.0
        03af5596         0.0             1.0
        041627fe         0.0             1.0
        045a01b7         0.0            12.0
        04706d48         0.0             7.0
        050f1a22         0.0            16.0
        064ec6ea         1.0            53.0
        06df5403         0.0            37.0
        072a96a1         0.0            12.0
        07535fbd         0.0             1.0
        07aabe67         0.0             1.0
        08343c0e         1.0             6.0
        0869b313         0.0             4.0
        0887367d         0.0           131.0
        09252ee6         0.0            73.0
        0a5879fc         1.0            30.0
        0aa463a0         0.0            18.0
        0ada2f89         0.0             3.0
...                      ...             ...
11      ff97d56f         0.0            57.0
        ff97e281         0.0             2.0
        ff9a58dd         0.0            70.0
        ff9b9954         0.0             3.0
        ff9da79b         0.0            20.0
        ff9dfd46         0.0            10.0
        ff9e0500         2.0            57.0
        ff9ebf5a         0.0             1.0
        ff9fc164         0.0            16.0
        ffa0b2f3         0.0            11.0
        ffa28549         0.0            16.0
        ffa9ac98         0.0            12.0
        ffacfff0         0.0             2.0
        ffb598de         0.0            11.0
        ffba4aec         0.0             1.0
        ffcf952b         0.0            53.0
        ffd62616         0.0             6.0
        ffd9756a         0.0             1.0
        ffda14ca         0.0             1.0
        ffdafc72         1.0            23.0
        ffe066cc         0.0            24.0
        ffe53446         2.0           110.0
        ffe7e848         0.0            19.0
        ffeb80c5         0.0             2.0
        ffed0342         0.0            57.0
        ffee0f18         0.0            60.0
        ffef83e6         0.0            34.0
        fff1659c         0.0            15.0
        fff1b11a         0.0            19.0
        fff1caee         0.0             1.0

[60532 rows x 2 columns]

In [85]:
def porcentaje(row):
    if (row['viewed product'] != 0):
        return row['conversion'] / row['viewed product']
    if (row['conversion'] != 0):
        return 1
    return 0

prod_vistos_por_conversion['promedio'] = prod_vistos_por_conversion.apply(porcentaje,axis=1)

In [86]:
prod_vistos_por_conversion

event             conversion  viewed product  promedio
periodo person                                        
1       00546f4c         0.0             8.0  0.000000
        006cabc5         0.0            36.0  0.000000
        009944e1         0.0            24.0  0.000000
        016b86b9         0.0            23.0  0.000000
        01716e05         0.0             1.0  0.000000
        01f061eb         0.0            14.0  0.000000
        022ca64b         0.0            12.0  0.000000
        027870af         1.0            28.0  0.035714
        0297fc1e         0.0            11.0  0.000000
        02a11379         0.0             1.0  0.000000
        02ac4fca         0.0             1.0  0.000000
        0318fd8d         0.0             5.0  0.000000
        03439225         0.0            33.0  0.000000
        03af5596         0.0             1.0  0.000000
        041627fe         0.0             1.0  0.000000
        045a01b7         0.0            12.0  0.000000
        04706d48         0.0             7.0  0.000000
        050f1a22         0.0            16.0  0.000000
        064ec6ea         1.0            53.0  0.018868
        06df5403         0.0            37.0  0.000000
        072a96a1         0.0            12.0  0.000000
        07535fbd         0.0             1.0  0.000000
        07aabe67         0.0             1.0  0.000000
        08343c0e         1.0             6.0  0.166667
        0869b313         0.0             4.0  0.000000
        0887367d         0.0           131.0  0.000000
        09252ee6         0.0            73.0  0.000000
        0a5879fc         1.0            30.0  0.033333
        0aa463a0         0.0            18.0  0.000000
        0ada2f89         0.0             3.0  0.000000
...                      ...             ...       ...
11      ff97d56f         0.0            57.0  0.000000
        ff97e281         0.0             2.0  0.000000
        ff9a58dd         0.0            70.0  0.000000
        ff9b9954         0.0             3.0  0.000000
        ff9da79b         0.0            20.0  0.000000
        ff9dfd46         0.0            10.0  0.000000
        ff9e0500         2.0            57.0  0.035088
        ff9ebf5a         0.0             1.0  0.000000
        ff9fc164         0.0            16.0  0.000000
        ffa0b2f3         0.0            11.0  0.000000
        ffa28549         0.0            16.0  0.000000
        ffa9ac98         0.0            12.0  0.000000
        ffacfff0         0.0             2.0  0.000000
        ffb598de         0.0            11.0  0.000000
        ffba4aec         0.0             1.0  0.000000
        ffcf952b         0.0            53.0  0.000000
        ffd62616         0.0             6.0  0.000000
        ffd9756a         0.0             1.0  0.000000
        ffda14ca         0.0             1.0  0.000000
        ffdafc72         1.0            23.0  0.043478
        ffe066cc         0.0            24.0  0.000000
        ffe53446         2.0           110.0  0.018182
        ffe7e848         0.0            19.0  0.000000
        ffeb80c5         0.0             2.0  0.000000
        ffed0342         0.0            57.0  0.000000
        ffee0f18         0.0            60.0  0.000000
        ffef83e6         0.0            34.0  0.000000
        fff1659c         0.0            15.0  0.000000
        fff1b11a         0.0            19.0  0.000000
        fff1caee         0.0             1.0  0.000000

[60532 rows x 3 columns]

In [87]:
prod_vistos_por_conversion = prod_vistos_por_conversion['promedio'].unstack(level=0)
prod_vistos_por_conversion.reset_index(inplace=True)
prod_vistos_por_conversion = prod_vistos_por_conversion.fillna(0)
prod_vistos_por_conversion

periodo,person,1,2,3,4,5,6,7,8,9,10,11
0,00091926,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
1,00091a7a,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
2,000ba417,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.008696
3,000c79fe,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
4,000e4d9e,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
5,000e619d,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
6,001001be,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.024390,0.000000
7,0010e89a,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
8,0016c4b5,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000
9,001802e4,0.0,0.0,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000,0.000000,0.000000


# Feature: returns por persona

In [28]:
returns_por_persona = events[['person','periodo','new_vs_returning']]
returns_por_persona = returns_por_persona.loc[(returns_por_persona['new_vs_returning']=='Returning')]
returns_por_persona

,person,periodo,new_vs_returning
2136629,4640420b,10,Returning
2136630,4640420b,10,Returning
2136631,4640420b,10,Returning
2136632,4640420b,10,Returning
2136633,4640420b,10,Returning
2136634,4640420b,10,Returning
2136635,4640420b,10,Returning
2136636,4640420b,10,Returning
2136637,4640420b,10,Returning
2136638,4640420b,11,Returning


In [29]:
returns_por_persona = returns_por_persona.groupby(['periodo','person'])['new_vs_returning'].count().unstack(level=0)
returns_por_persona = returns_por_persona.fillna(0)
returns_por_persona

periodo,1,2,3,4,5,6,7,8,9,10,11
person,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
00091926,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,20.0,10.0
000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0
000e4d9e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,10.0
000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
001001be,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
0019e639,0.0,0.0,2.0,3.0,0.0,0.0,0.0,0.0,5.0,6.0,2.0
001ca5ee,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,2.0,2.0
001dfc31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0


In [30]:
returns_por_persona.reset_index(inplace=True)
#returns_por_persona  = returns_por_persona[['person',7,8,9,10,11]]

In [31]:
returns_por_persona['returns_por_persona_mean'] = returns_por_persona.mean(axis=1)

In [33]:
returns_por_persona_mean = returns_por_persona[['person','returns_por_persona_mean']]

In [38]:
train_data = train_data.merge(returns_por_persona_mean,on='person',how='left')
train_data['returns_por_persona_mean'].fillna(0,inplace=True)

In [44]:
test_kaggle = test_kaggle.merge(returns_por_persona_mean,on='person',how='left')
test_kaggle['returns_por_persona_mean'].fillna(0,inplace=True)

In [46]:
test_kaggle.to_csv('test_kaggle.csv')
train_data.to_csv('train_data.csv')

# Feature: motor de busqueda

In [43]:
palabra_buscada = events[['person','search_term','event','periodo']]

In [44]:
palabra_buscada = palabra_buscada.loc[palabra_buscada['event']=='searched products']

In [45]:
palabra_buscada = palabra_buscada.groupby(['periodo','person','search_term'])['event'].count().unstack(level=0)

In [46]:
palabra_buscada.reset_index(inplace=True)

In [56]:
palabra_buscada['busca_palabra_popular'] = (palabra_buscada['search_term'].str.lower().str.contains('iphone')) | (palabra_buscada['search_term'].str.lower().str.contains('plus')) | (palabra_buscada['search_term'].str.lower().str.contains('galaxy'))
palabra_buscada = palabra_buscada.loc[palabra_buscada['busca_palabra_popular']]
palabra_buscada = palabra_buscada.fillna(0)

In [57]:
for x in range(1,12):
    palabra_buscada[x] = palabra_buscada[x].apply(lambda a: 1 if a != 0 else a)
palabra_buscada

periodo,person,search_term,1,2,3,4,5,6,7,8,9,10,11,busca_palabra_popular
0,000c79fe,Galaxy a8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
1,000c79fe,Galaxy s8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
2,000c79fe,Iphone 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
3,000c79fe,Iphone 6s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
4,000c79fe,Iphone 7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
5,000c79fe,Iphone 7s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
6,000c79fe,Iphone x,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
10,001001be,IPhone 6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,True
11,001802e4,Aiphone 6s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,True
13,0019e639,Samsung Galaxy Win Duos,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True


In [17]:
palabra_buscada = palabra_buscada[['person',7,8,9,10,11]]

In [59]:
usuarios_busca_palabra_popular = palabra_buscada.loc[palabra_buscada['busca_palabra_popular']==True]
usuarios_busca_palabra_popular = usuarios_busca_palabra_popular[['person','busca_palabra_popular']]
usuarios_busca_palabra_popular

periodo,person,busca_palabra_popular
0,000c79fe,True
1,000c79fe,True
2,000c79fe,True
3,000c79fe,True
4,000c79fe,True
5,000c79fe,True
6,000c79fe,True
10,001001be,True
11,001802e4,True
13,0019e639,True


In [409]:
train_data=train_data.merge(usuarios_busca_palabra_popular,on='person',how='left')
train_data['usuarios_busca_palabra_popular'].fillna(False,inplace=True)

In [411]:
test_kaggle = test_kaggle.merge(usuarios_busca_palabra_popular,on='person',how='left')
test_kaggle['usuarios_busca_palabra_popular'].fillna(False,inplace=True)

In [415]:
test_kaggle.drop_duplicates('person',inplace=True)

In [416]:
train_data.to_csv('train_data.csv')
test_kaggle.to_csv('test_kaggle.csv')